In [1]:
import datetime as dt
import pandas as pd
import util as ut
import numpy as np
import random
import QLearner
import sys
import warnings

In [2]:
 #!/usr/bin/env python -W ignore::DeprecationWarning
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
sd=dt.datetime(2008,1,1)
ed=dt.datetime(2009,1,1)
symbol="IBM"


In [4]:
def getIndicators(symbol = "IBM", \
    sd=dt.datetime(2008,1,1), \
    ed=dt.datetime(2009,1,1), \
    sv = 10000):


    # go to 1 year back for learning
    sd = sd - dt.timedelta(365)

    symbols = [symbol]

    # get the data using util
    adj_close = ut.get_data(symbols, pd.date_range(sd, ed))
    adj_close.fillna(method='ffill', inplace=True)
    adj_close.fillna(method='bfill', inplace=True)

    # rolling mean
    rol_mean_20 = pd.rolling_mean(adj_close[symbol], window=20)
    rol_mean_150 = pd.rolling_mean(adj_close[symbol], window=150)
    df = adj_close[symbols]
    df['150-SMA'] = rol_mean_150
    df['20-SMA'] = rol_mean_20

    df_back_to_future = df[sd + dt.timedelta(365) : ]
    df_back_to_future = df_back_to_future / df_back_to_future.IBM[0]


    strategy = pd.DataFrame(index=df_back_to_future.index)
    strategy['moving_average'] = df_back_to_future['20-SMA'] - df_back_to_future['150-SMA']


    ## SLOW STOCHASTIC
    high = pd.rolling_max(adj_close[symbol], window=14)
    low = pd.rolling_min(adj_close[symbol], window=14)
    K = (adj_close[symbol] - low) / (high-low) * 100
    slow_stoch = pd.rolling_mean(K, window=3)
    # print slow_stoch
    slow_future = slow_stoch[sd+dt.timedelta(365):]
    strategy['slow_stoch'] = slow_future

    ### MACD
    e = 12
    rol_mean_e = pd.rolling_mean(adj_close[symbol], window=e)
    ema_12 = np.zeros(len(rol_mean_e.index))
    multiplier = 2./(e+1)
    ema_12[e-1] = rol_mean_e[e-1]
    for i in range(e, len(adj_close.index)):
        close = adj_close[symbol].values[i]
        em = ema_12[i-1]
        ema_12[i] = (close-em) * multiplier + em

    e = 26
    rol_mean_e = pd.rolling_mean(adj_close[symbol], window=e)
    ema_26 = np.zeros(len(rol_mean_e.index))
    multiplier = 2./(e+1)
    ema_26[e-1] = rol_mean_e[e-1]
    for i in range(e, len(adj_close.index)):
        close = adj_close[symbol].values[i]
        em = ema_26[i-1]
        ema_26[i] = (close-em) * multiplier + em

    df_macd = pd.DataFrame(rol_mean_e)
    df_macd['ema_12'] = ema_12
    df_macd['ema_26'] = ema_26
    df_macd['MACD'] = df_macd['ema_12'] - df_macd['ema_26']
    MACD = ema_12 - ema_26
    MACD[:25] = 0

    ## SIGNAL line
    e = 9
    rm_macd = pd.rolling_mean(MACD, window=e)
    sig = np.zeros(len(MACD))
    multiplier = 2./(e+1)
    sig[e-1] = rm_macd[e-1]
    for i in range(e, len(MACD)):
        close = MACD[i]
        em = sig[i-1]
        sig[i] = (close - em) * multiplier + em

    df_macd['signal'] = sig
    df_macd['hist'] = df_macd['MACD'] - df_macd['signal']

    strategy['MACD'] = df_macd[sd + dt.timedelta(365):]['hist']
    return strategy


In [7]:
symbol = "IBM"
sd = dt.datetime(2008, 1, 1)
ed = dt.datetime(2010,1,1)
sv =10000

indicators = getIndicators(symbol, sd, ed, sv)


In [8]:
indicators

,moving_average,slow_stoch,MACD
2008-01-02,-0.019910,43.440531,-0.081211
2008-01-03,-0.020671,19.142420,-0.274822
2008-01-04,-0.023734,2.450230,-0.604944
2008-01-07,-0.027996,1.735579,-0.843677
2008-01-08,-0.032901,0.000000,-1.091740
2008-01-09,-0.037979,1.704105,-1.143811
2008-01-10,-0.041280,7.229538,-1.015371
2008-01-11,-0.046137,7.436096,-1.004123
2008-01-14,-0.048672,18.409502,-0.624926
2008-01-15,-0.050563,23.016571,-0.406255


In [9]:
q_learner = QLearner.QLearner(num_states=1000, num_actions=3, alpha=0.2, gamma=0.95, rar=0.5, radr=0.99, dyna=0, verbose=False)

In [10]:
# for i in range(len(indicators.index)):
a = (indicators[indicators.columns[0]].values)
b = (indicators[indicators.columns[1]].values)
c = (indicators[indicators.columns[2]].values)
c

array([ -8.12111539e-02,  -2.74821907e-01,  -6.04944324e-01,
        -8.43677447e-01,  -1.09174027e+00,  -1.14381102e+00,
        -1.01537066e+00,  -1.00412276e+00,  -6.24926172e-01,
        -4.06255403e-01,  -2.46405769e-01,  -1.49749178e-01,
         6.78613962e-02,   8.74308107e-02,   3.95185191e-01,
         6.27856561e-01,   6.12853571e-01,   6.08566857e-01,
         6.48763342e-01,   6.20719592e-01,   6.62278628e-01,
         7.73769584e-01,   7.38965875e-01,   5.09223105e-01,
         2.77945563e-01,   4.37948032e-02,  -5.10704007e-02,
         1.33846489e-03,   1.15863859e-01,   2.91772950e-01,
         2.50571694e-01,   2.11448412e-01,   1.04997108e-01,
         1.97174127e-01,   1.87118766e-01,   2.34978504e-01,
         3.66509810e-01,   6.77409665e-01,   9.52704380e-01,
         9.95073974e-01,   8.76750731e-01,   7.65297354e-01,
         7.27980371e-01,   6.30852149e-01,   3.50555850e-01,
         2.21427729e-01,   1.13541244e-01,   1.66678799e-01,
         2.05258562e-01,

In [11]:
mov_av = pd.qcut(a, 10, labels=False)
slow_st = pd.qcut(b, 10, labels=False)
macd = pd.qcut(c, 10, labels=False)
disc = mov_av*100+slow_st*10+macd
disc

array([334, 313, 300, 300, 300, 200, 200, 200, 210, 222, 223, 223, 235,
       235, 257, 168, 188, 178, 168, 178, 188, 189, 189, 168, 247, 225,
       224, 225, 235, 257, 257, 256, 245, 246, 256, 376, 377, 398, 399,
       399, 379, 369, 379, 378, 377, 366, 355, 376, 386, 386, 365, 354,
       365, 465, 485, 485, 485, 474, 443, 431, 411, 421, 411, 421, 411,
       422, 422, 432, 444, 443, 443, 443, 455, 477, 498, 498, 498, 588,
       588, 587, 566, 565, 554, 564, 564, 553, 643, 654, 674, 684, 685,
       785, 796, 797, 797, 776, 755, 743, 732, 722, 833, 854, 885, 896,
       975, 964, 954, 954, 942, 932, 921, 920, 810, 811, 832, 832, 821,
       822, 711, 711, 701, 722, 611, 610, 500, 500, 500, 510, 522, 534,
       544, 445, 446, 446, 446, 467, 488, 499, 489, 489, 489, 499, 489,
       567, 557, 557, 566, 555, 544, 545, 555, 565, 575, 544, 523, 512,
       512, 522, 521, 510, 500, 400, 411, 411, 411, 401, 413, 412, 411,
       400, 300, 300, 300, 310, 321, 323, 334, 323, 323, 302, 31